# Notebook for own analysis

In [1]:
import json
import pathlib

import cv2
import numpy as np
import matplotlib.pyplot as plt

from statistics import mode
from typing import Any, Callable

from src import video, feed
from src.display import showarray
from src.processing import ImageMatcher, Homographer, BoardMapper
from src.processing import analyzers, detectors, features, calc
from src.processing import events
from src.processing.context import ContextReader
from src.monopoly.elements import FieldIndex, deserialize_families, deserialize_fields

### Misc Functions

In [2]:
board_image = cv2.imread("data/images/board.jpg")

CARD_RATIO = 8.5 / 5.5

def filter_contours(contours: list[np.ndarray]) -> list[np.ndarray]:
    
    return [
        contour 
        for contour in contours
        if calc.is_min_area_in_bounds_and_good_edges_ratio(contour, 10_000, 25_000, CARD_RATIO)
    ]

serialized_families = json.loads(pathlib.Path("data/meta/families.json").read_text())
serialized_fields = json.loads(pathlib.Path("data/meta/fields.json").read_text())

families = {family.name: family for family in deserialize_families(serialized_families)}
fields = list(deserialize_fields(serialized_fields, families))
FIELD_INDEX = FieldIndex(fields)

def field_id_to_name(field_id):
    if field_id is None:
        return "Searching..."
    field = FIELD_INDEX.get_by_id(field_id)
    if field is not None:
        return field.name
    return "Searching..."

def decode_position(field_index: FieldIndex, position: tuple[int, int] | None, board_mapper: BoardMapper, last_decoded: int | None = None):
    decoded_position = None
    if position is not None:
        section, index = board_mapper.map_point_to_field_position(*position)
        if section not in ["inside", "outside"]:
            field = field_index.get_by_place(section, index)
            if field is not None:
                decoded_position = field.id
    if decoded_position is None:
        decoded_position = last_decoded
    return decoded_position


## Live feed processor - shows the detection results as it reads through the video

Specify path to video for it to work!

In [17]:
PATH_TO_VIDEO = ... # Provide path to your video here!

board_drawing_entry = analyzers.PointsDrawingEntry("board_points", (255, 255, 0))
dice_area_drawing_entry = analyzers.RectDrawingEntry("dice_area_rect", (0, 255, 255))

rect_analyzer = analyzers.RectDrawingAnalyzer(
    [board_drawing_entry, dice_area_drawing_entry]
)

white_piece_analyzer = analyzers.WhitePieceAnalyzer(
    piece_detector=detectors.get_white_piece_detector(),
    binarizer=features.get_clear_inverted_edges,
    threaded=True
)
black_piece_analyzer = analyzers.BlackPieceAnalyzer(
    piece_detector=detectors.get_black_piece_detector(),
    binarizer=features.get_clear_inverted_edges, 
    threaded=True
)

dots_analyzer = analyzers.DotsAnalyzer(blob_detector=detectors.get_dots_detector(), threaded=True)
cards_analyzer = analyzers.CardsAnalyzer(
    edge_detector=features.get_threshold_edges,
    contour_filter=filter_contours,
    threaded=True
)

live_feed = feed.LiveFrameProcessor("feed", 800, 400)

# Change white/black cards context callback in case sides or orientation are swapped / changed
context_readers = [
    ContextReader("black_pos", 130, lambda context: decode_position(FIELD_INDEX, context["black_pos"], context["board_mapper"]), field_id_to_name),
    ContextReader("white_pos", 130, lambda context: decode_position(FIELD_INDEX, context["white_pos"], context["board_mapper"]), field_id_to_name),
    ContextReader("dice_dots", 80, lambda context: len(context["dice_dots"])), 
    ContextReader("dice_pos_x", 30, lambda context: np.average([kp.pt[0] for kp in context["dice_dots"]]) if context["dice_dots"] else 0 ), 
    ContextReader("dice_pos_y", 30, lambda context: np.average([kp.pt[1] for kp in context["dice_dots"]]) if context["dice_dots"] else 0 ), 
    ContextReader("black_cards", 130, lambda context: len([card for card in context["cards"] if card[0] >= context["width"] / 2])),
    ContextReader("white_cards", 130, lambda context: len([card for card in context["cards"] if card[0] < context["width"] / 2])),
    ContextReader("card_dots", 200, lambda context: len(context["card_dots"])),
]

event_detectors = [
    events.DiceEventDetector(context_readers),
    events.CardEventDetector(context_readers, "black"),
    events.CardEventDetector(context_readers, "white"),
    events.MoveEventDetector(context_readers, "black"),
    events.MoveEventDetector(context_readers, "white"),
]

events_analyzer = analyzers.EventsAnalyzer(context_readers=context_readers, event_detectors=event_detectors)
used_analyzers_with_events = [rect_analyzer, white_piece_analyzer, black_piece_analyzer, dots_analyzer, cards_analyzer, events_analyzer]

with video.VideoHandler(PATH_TO_VIDEO) as vh:
    first_frame = vh.get_frame(0)
    image_matcher = ImageMatcher(first_frame, board_image)
    matches = image_matcher.get_matches()
    homographer = Homographer(image_matcher.kp_ref, image_matcher.kp_match, matches)
    inv_homography, _ = homographer.get_inverse_homography()
    homography, _ = homographer.get_homography()
    board_mapper = BoardMapper(homography, max(board_image.shape))
    dst_rect = features.get_board_min_rect_from_homography(inv_homography, board_image)
    board_points = cv2.boxPoints(dst_rect)
    dice_area_rect = features.find_dice_throwing_rect(first_frame)
    if dice_area_rect is None:
        dice_area_rect = features.find_dice_throwing_rect(first_frame, 80, 30)
    additonal_context = dict(
        board_points=board_points,
        dice_area_rect=dice_area_rect, 
        board_mapper=board_mapper,
        height=first_frame.shape[0],
        width=first_frame.shape[1],
    )
    vh.go_through_video(live_feed, used_analyzers_with_events, additional_context=additonal_context)

MOVE EVENT
DICE EVENT
DICE EVENT
MOVE EVENT
MOVE EVENT
PROPERTY EVENT


In [12]:
name = "Easy_2"
with video.VideoHandler(f"data/recordings/{name}.mp4") as vh:
    vh.go_through_video(live_feed, frame_analyzers=[])